#### **Импорт библиотек**

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import os
import time
import logging
import re


#### **Определение класса для скачивание файлов pdf с единичной страницы html**


In [2]:
class PDFDownloader:
    def __init__(self, link_page, download_directory):
        self.link_page = link_page # Задание адреса страницы
        self.download_directory = download_directory # Задание каталога загрузки
        self.setup_logging() # Настройка логирования
        self.driver = self.setup_driver() # Инициализация драйвера Chrome
        self.link_selector = 'a[href$=".pdf"], a[href$=".docx"]'  # Селектор для поиска ссылок на файлы

            
    def setup_logging(self): # Настройка логирования для сохранения в файл
        # Убедимся, что root_logger не имеет других обработчиков
        for handler in logging.root.handlers[:]:
            logging.root.removeHandler(handler)
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger()
        file_handler = logging.FileHandler('download_log.log', mode='a')  # 'a' для добавления (append)
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)
        self.logger.addHandler(file_handler)  # Добавление обработчика логирования

    
    def setup_driver(self):
        chrome_options = Options() # Создание объекта Options для настройки параметров браузера Chrome
        #chrome_options.add_argument("--no-sandbox") # Отключение режима песочницы для повышения производительности при работе в контейнерах
        #chrome_options.add_argument("--disable-dev-shm-usage")  # Отключение использования общей памяти (shm), что помогает избежать ошибок в контейнерах
        chrome_options.add_argument("--disable-gpu") # Отключение аппаратного ускорения для графики, что может улучшить совместимость
        chrome_options.add_argument("--window-size=1920,1080") # Установка размера окна браузера
        chrome_options.add_experimental_option('prefs', {
            "download.default_directory": self.download_directory, # Установка пути к директории, в которую будут загружаться файлы
            "download.prompt_for_download": False, # Отключение диалогового окна запроса на загрузку файлов
            "download.directory_upgrade": True, # Разрешение на обновление пути к директории загрузки
            "plugins.always_open_pdf_externally": True # Автоматическая загрузка PDF-файлов без предварительного просмотра в браузере
        })
        # chrome_options.add_argument("--headless")  # Запуска Chrome без GUI
        return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    

    # Функция проверки наличия незавершенных закачек в папке скачивания        
    def download_complete(self, timeout=30):
        """Ожидает окончания загрузки всех файлов в папке загрузок, прерывается после заданного таймаута."""
        start_time = time.time()
        while True:
            if all(not (filename.endswith(".crdownload") or filename.endswith(".tmp")) for filename in os.listdir(self.download_directory)):
                break # выход из цикла, если нет временных файлов 
            time.sleep(1) # увеличение времени ожидания между проверками
            elapsed_time = time.time() - start_time
            if elapsed_time > timeout:
                self.logger.warning("Timeout reached while waiting for downloads to finish.")
                break

    
    # Функция проверяет наличие файла с учётом возможных суффиксов, используя регулярные выражения:
    def file_exists(self, filename):
        """ Проверяем, существует ли файл с этим именем (учитывая возможные суффиксы). """
        base_name, ext = os.path.splitext(filename)
        pattern = re.compile(re.escape(base_name) + r'(\(\d+\))?' + re.escape(ext) + r'$')
        for file in os.listdir(self.download_directory):
            if pattern.match(file):
                return True
        return False


    # Функция удаляет временные файлы из папки загрузки 
    def clean_up_temp_files(self):
        for file in os.listdir(self.download_directory):
            if file.endswith(".tmp"):
                try:
                    os.remove(os.path.join(self.download_directory, file))
                    self.logger.info(f"Temporary file .tmp {file} removed.")
                except Exception as e:
                    self.logger.error(f"Failed to remove .tmp {file}: {e}")
        for file in os.listdir(self.download_directory):
            if file.endswith(".crdownload"):
                try:
                    os.remove(os.path.join(self.download_directory, file))
                    self.logger.info(f"Temporary file .crdownload {file} removed.")
                except Exception as e:
                    self.logger.error(f"Failed to remove .crdownload {file}: {e}")
                    
        
    def download_files(self):
        """Загружает файлы с веб-страницы."""
        try:
            self.driver.get(self.link_page)  # Загрузка веб-страницы
            links = [link.get_attribute('href') for link in self.driver.find_elements(By.CSS_SELECTOR, 'a[href$=".pdf"], a[href$=".docx"]')] # получение полных ссылок скачиваемых файлов
            self.logger.info(f"Page loaded, number of files to download: {len(links)} PS.")
            for href in links:
                pdf_name = href.split('/')[-1]
                if not self.file_exists(pdf_name): # при отсутствии дубликатов файлов происходит скачивание
                    self.driver.get(href)
                    self.download_complete()
                    self.logger.info(f'Downloaded: {pdf_name}')
                else:   # при наличии дубликатов сохраняет в лог что файл уже скачан
                    self.logger.info(f'Already downloaded: {pdf_name}')
        except Exception as e:
            self.logger.error(f'Error during downloading: {e}')
            self.clean_up_temp_files()
        finally:
            self.driver.quit()
            self.clean_up_temp_files()
            self.logger.info('Driver closed.')
    

    
    # Закрытие и удаление обработчика
    def __del__(self):
        """Закрытие и удаление ресурсов перед уничтожением объекта."""
        self.driver.quit()
        for handler in self.logger.handlers[:]:
            handler.close()
            self.logger.removeHandler(handler)


#### **Скачивание файлов pdf со страницы**

In [3]:
# Example usage:
# if __name__ == '__main__':
#    downloader = PDFDownloader('https://files.stroyinf.ru/cat0/205-1.htm', r"C:\Users\User\Documents\MFTI_VKR\__parsing\1")
#    downloader.download_files()
    
if __name__ == '__main__':
    base_url = "https://files.stroyinf.ru/cat0/205-{}.htm"
    download_directory = r"C:\Users\User\Documents\MFTI_VKR\__parsing\1"
    for i in range(8):  # От 0 до 7 включительно
        url = base_url.format(i)  # Формирование URL для каждой страницы
        print(f"Downloading from: {url}")  # Опционально: вывод информации о текущей странице
        downloader = PDFDownloader(url, download_directory)
        downloader.download_files()
        print("Download complete for: ", url)  # Опционально: подтверждение завершения загрузки

        # Опционально: Очистка или перезапуск драйвера, если необходимо
        # Это может быть полезно, если в процессе работы драйвера возникают утечки памяти или другие проблемы
        # Некоторые приложения могут требовать перезапуск драйвера для освобождения ресурсов
        downloader.driver.quit()  # Закрытие драйвера
        print("Driver closed for cleanup.")  # Опционально: информация о закрытии драйвера



INFO:WDM:====== WebDriver manager ======


INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [C:\Users\User\.wdm\drivers\chromedriver\win64\124.0.6367.91\chromedriver-win32/chromedriver.exe] found in cache
INFO:root:Page loaded, number of files to download: 10 PS.
INFO:root:Already downloaded: 51176.pdf
INFO:root:Already downloaded: 51247.pdf
INFO:root:Already downloaded: 51254.pdf
INFO:root:Already downloaded: 51442.pdf
INFO:root:Already downloaded: 51415.pdf
INFO:root:Already downloaded: 51413.pdf
INFO:root:Already downloaded: 51411.pdf
INFO:root:Already downloaded: 53549.pdf
INFO:root:Already downloaded: 53547.pdf
INFO:root:Already downloaded: 53544.pdf
INFO:root:Driver closed.


Download complete for:  https://files.stroyinf.ru/cat0/205-0.htm


INFO:WDM:====== WebDriver manager ======


Driver closed for cleanup.


...

Download complete for:  https://files.stroyinf.ru/cat0/205-6.htm


INFO:WDM:====== WebDriver manager ======


Driver closed for cleanup.


INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [C:\Users\User\.wdm\drivers\chromedriver\win64\124.0.6367.91\chromedriver-win32/chromedriver.exe] found in cache
Timeout reached while waiting for downloads to finish.


Download complete for:  https://files.stroyinf.ru/cat0/205-7.htm
Driver closed for cleanup.
